In [1]:
import numpy as np
import matplotlib.pyplot as plt 
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.animation as animation
import time

def setGraph( N, edgeList ):
    W = np.zeros((N,N))
    for e in edgeList:
        i = e[0]
        j = e[1]
        W[i][j] = 1
        W[j][i] = 1       
    return W

def drawLines(k, edgeList, x, x_inf):
    plt.cla()
    for i in range(N):
        ax.scatter(x[3*i,0],x[3*i+1,0],x[3*i+2,0],c='blue') # initial position
        ax.scatter(x[3*i,K-1],x[3*i+1,K-1],x[3*i+2,K-1],c='red') # current position
        ax.scatter(x_inf[3*i],x_inf[3*i+1],x_inf[3*i+2],c='green') # final position
        ax.plot(x[3*i,:],x[3*i+1,:],x[3*i+2,:])    
    for e in edgeList:
        i = e[0]
        j = e[1]
        ax.plot([x[3*i,k-1], x[3*j,k-1]], [x[3*i+1,k-1], x[3*j+1,k-1]], [x[3*i+2,k-1], x[3*j+2,k-1]], c='black')

def setIncidence( N, edgeList ):
    m = len(edgeList)
    B = np.zeros((N,m))
    for idx, e in enumerate(edgeList):
        i = e[0]
        j = e[1]
        B[i,idx] = 1
        B[j,idx] = -1
    return B

if __name__ == '__main__':

    n = 3 # dimension
    N = 8 # number of robots
    K = 2000 # number of iterations
    eps = 0.02# discretization constant
    
    kp = 1
    kv = 1*kp
    kap = 0.2*kp
    kav = 1.8*kp
    p = np.zeros((n*N, K)) 
    v = np.zeros((n*N, K)) 
    x = np.concatenate((p, v), axis=0)
    
    p_inf = np.array([0,0,0, 0,1,0, 1,1,0, 1,0,0, 0,0,1, 0,1,1, 1,1,1, 1,0,1]).T+2.0
    v_inf = np.zeros(n*N)
    p_init = 5*(np.random.rand(n*N)-0.5)
    v_init = 0.5*v_inf
    x[:,0] = np.concatenate((p_init, v_init), axis=0)
    
    x_inf = np.concatenate((p_inf, v_inf),axis=0) 
                            

In [2]:
    edgeList = [(0,1),
                (1,2),
                (2,3),
                (3,0),
                (4,5),
                (5,6),
                (6,7),
                (7,4),
                (0,4),
                (1,5),
                (2,6),
                (3,7),]
    W = setGraph(N, edgeList)
    Dout = np.diag(np.sum(W, axis=1))
    L = Dout - W 
    B = setIncidence(N,edgeList)
    B_kron = np.kron(B,np.identity(n))
    
    G_pos = np.diag(np.array([1, 1, 0, 0, 0, 1, 0, 0]))
    G_vel = np.diag(np.array([1, 1, 1, 1, 1, 1, 1, 1]))
    
    A11 = np.zeros((n*N,n*N))
    A12 = np.identity(n*N)
    A21 = -kp*np.kron(L, np.identity(n)) - kap*np.kron(G_pos, np.identity(n))
    A22 = -kv*np.kron(L, np.identity(n)) - kav*np.kron(G_vel, np.identity(n))
    A = np.concatenate((A11,A12), axis=1)
    A = np.concatenate((A, np.concatenate((A21,A22), axis=1)), axis=0)

In [3]:
    for k in range(K-1):
        x[:,k+1] = x[:,k] + eps*np.dot(A,x[:,k]-x_inf) + eps*0.01*(np.random.rand(2*n*N)-0.5)

    print(np.linalg.norm(x[0:15:1,K-1]-x_inf[0:15:1]))
    print(np.linalg.norm(x[15:30:1,K-1]))


2.04199998252
6.85405153776


In [4]:
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    
    ani = animation.FuncAnimation(fig, drawLines, fargs=[edgeList, x, x_inf], frames=np.arange(1,K-1,10), interval=10)   
    plt.show()

AttributeError: 'NoneType' object has no attribute 'tk'